# melt enhancement

### New features
* Melts different groups of columns by passing a list of lists into value_vars. Each group gets melted into its own column. This feature replaces the need for `lreshape`.
* When melting different groups of columns, groups do not have to be the same length. The shorter groups are filled with missing values.
* Adds parameters `stubnames`(boolean), `prefix` and `sep` from function `wide_to_long`. It keeps the suffixes in separate columns and does not align them like `wide_to_long`.
* Fixes bug when selecting a column level of a MultiIndex
* Can select any number of MultiIndex levels
* Works with repeated column names, which normally show up when selecting a subset of MultiIndex levels
* Performance is ~30-40% faster than original melt, slightly slower than `lreshape` and much faster than `wide_to_long` 

### How to use new melt
Run this code block below and directly use the function `melt`. Put `pd.` in front of call to `melt` to use current pandas version.

In [1]:
from pandas.core.dtypes.common import is_list_like
from pandas.core.frame import DataFrame
from pandas.core.index import MultiIndex
from pandas import compat
from IPython.display import display
from pandas.core.reshape.concat import concat
import re
from pandas.core.tools.numeric import to_numeric
from pandas.util._decorators import Appender
from pandas.core.frame import _shared_docs
import numpy as np
import pandas as pd
import pandas.util.testing as tm


def _melt(frame, id_vars=None, value_vars=None, var_name=None,
          value_name='value', col_level=None, stubnames=False,
          suffix=r'\d+', sep='', extra_group=0, var_end=None):
    # TODO: what about the existing index?

    def check_vars(frame, var, var_string):
        for v in var:
            if num_col_levels > 1:
                if not isinstance(v, tuple):
                    raise ValueError('{} must be a list of tuples'
                                     ' when columns are a MultiIndex'
                                     .format(var_string))
                elif len(v) != num_col_levels:
                    raise ValueError('all tuples in {} must be length {}'
                                     .format(var_string,
                                             frame.columns.nlevels))
            else:
                if is_list_like(v) and len(v) > 1:
                    raise ValueError('DataFrame has only a single level of '
                                     'columns. {} is not a column'.format(v))

    if len(col_level) == 0:
        num_col_levels = frame.columns.nlevels
    else:
        num_col_levels = len(col_level)

    check_vars(frame, id_vars, 'id_vars')
    check_vars(frame, value_vars, 'value_vars')

    if var_name != [] and len(var_name) != num_col_levels:
        raise ValueError('Length of var_name must match effective number of '
                         'column levels.')
    
    if col_level != []:
        droplevels = list(range(frame.columns.nlevels))
        for level in col_level:
            if isinstance(level, int):
                droplevels.remove(level)
            else:
                droplevels.remove(frame.columns.names.index(level))
        if droplevels != []:
            frame = frame.copy()
            frame.columns = frame.columns.droplevel(droplevels)

    if stubnames and isinstance(frame.columns, MultiIndex):
        raise ValueError('Stubnames only work with single-index DataFrames')
        
    for iv in id_vars:
        if iv not in frame.columns:
            raise KeyError('{} not in columns'.format(iv))

    if value_vars != []:
        for vv in value_vars:
            if vv not in frame.columns:
                raise KeyError('{} not in columns'.format(vv))
                
    if var_name == []:
        names = list(frame.columns.names)
        if len(names) == 1:
            if names[0] is None:
                var_name.append('variable')
            else:
                var_name.append(names[0])
        elif names.count(None) == 1:
            names[names.index(None)] = 'variable'
            var_name = names
        else:
            missing_name_count = 0
            for name in names:
                if name is None:
                    var_name.append('variable_{}'.format(missing_name_count))
                    missing_name_count += 1
                else:
                    var_name.append(name)
    if var_end is not None:
        var_name = [vn + '_' + str(var_end) for vn in var_name]
    
    N = len(frame)
    
    non_id_ilocs = []
    if value_vars != []:
        for v in value_vars:
            for i, v1 in enumerate(frame.columns):
                if v == v1:
                    non_id_ilocs.append(i)
    else:
        if id_vars == []:
            non_id_ilocs = list(range(frame.shape[1]))
        else:
            for i, v in enumerate(frame.columns):
                if v not in id_vars:
                    non_id_ilocs.append(i)
                        
    K = len(non_id_ilocs)

    mdata = {}
    mcolumns = []
    for col in id_vars:
        pandas_obj = frame[col]
        if isinstance(pandas_obj, DataFrame):
            for i in range(pandas_obj.shape[1]):
                col_name = col + '_id_' + str(i)
                mdata[col_name] = np.tile(pandas_obj.iloc[:, i].values, K + extra_group)
                mcolumns.append(col_name)
        else:
            mdata[col] = np.tile(pandas_obj, K + extra_group)
            mcolumns.append(col)

    values = np.concatenate([frame.iloc[:, i] for i in non_id_ilocs])
    if extra_group > 0:
        values = np.concatenate((values, np.full([N * extra_group], np.nan)))
    mdata[value_name[0]] = values
    
    for i, col in enumerate(var_name):
        values = frame.columns[non_id_ilocs]._get_level_values(i)
        if stubnames:
            regex = '^{0}{1}'.format(re.escape(value_name[0]), re.escape(sep))
            values = to_numeric(values.str.replace(regex, ''), errors='ignore')
        if isinstance(values, MultiIndex):
            # asanyarray will keep the columns as an Index
            values = np.asanyarray(values).repeat(N)
        else: 
            data_list = []
            for v in values.tolist():
                data_list.extend([v] * N)
            values = data_list
        if extra_group > 0:
            values = np.concatenate((values, np.full([N * extra_group], np.nan)))
        mdata[col] = values
    mcolumns += var_name + value_name
    
    return mdata, mcolumns


@Appender(_shared_docs['melt'] %
          dict(caller='pd.melt(df, ',
               versionadded="",
               other='DataFrame.melt'))
def melt(frame, id_vars=None, value_vars=None, var_name=None,
         value_name='value', col_level=None, stubnames=False,
         suffix=r'\d+', sep=''):
    def convert_to_list(val):
        if val is None:
            return []
        elif not is_list_like(val):
            return [val]
        else:
            return list(val)

    def get_var_names(df, stub, sep, suffix):
        regex = '^{0}{1}{2}$'.format(re.escape(stub), re.escape(sep), suffix)
        col_return = [col for col in df.columns if re.match(regex, col)]
        if col_return == []:
            raise ValueError('No stubname {}'.format(stub))
        return col_return

    id_vars = convert_to_list(id_vars)
    value_vars = convert_to_list(value_vars)
    var_name = convert_to_list(var_name)
    value_name = convert_to_list(value_name)
    col_level = convert_to_list(col_level)

    if stubnames:
        if value_vars == []:
            raise ValueError('Must provide stubnames as a list to value_vars')
        value_name = value_vars
        value_vars = [get_var_names(frame, stub, sep, suffix)
                      for stub in value_vars]
        if var_name == []:
            var_name = ['variable_' + v for v in value_name]

    if value_vars != [] and isinstance(value_vars[0], list):
        if var_name != []:
            if len(value_vars) != len(var_name):
                raise ValueError('Number of inner lists of value_vars must '
                                 'equal length of var_name '
                                 '{} != {}'.format(len(value_vars),
                                                   len(var_name)))
        else:
            var_name = [[]] * len(value_vars)

        if len(value_name) > 1:
            if len(value_vars) != len(value_name):
                raise ValueError('Number of inner lists of value_vars must '
                                 'equal length of value_name '
                                 '{} != {}'.format(len(value_vars),
                                                   len(value_name)))
        elif not stubnames:
            value_name = [value_name[0] + '_' + str(i) for i in range(len(value_vars))]

        value_vars_length = []
        for vv in value_vars:
            count = 0
            for col in frame.columns.values:
                if col in vv:
                    count += 1
            value_vars_length.append(count)
        max_group_len = max(value_vars_length)  

        mdata_list = []
        mcolumns_list = []
        vars_zipped = zip(value_vars, var_name, value_name, value_vars_length)
        for i, (val_v, var_n, val_n, vvl) in enumerate(vars_zipped):
            var_n = convert_to_list(var_n)
            val_n = convert_to_list(val_n)

            id_vars_ = [] if i > 0 else id_vars
            var_end = i if var_n == [] else None
            
            md, mc = _melt(frame, id_vars=id_vars_, value_vars=val_v,
                       var_name=var_n, value_name=val_n,
                       col_level=col_level, stubnames=stubnames,
                       suffix=suffix, sep=sep, 
                       extra_group=max_group_len - vvl,
                       var_end=var_end)

            mdata_list.append(md)
            mcolumns_list.append(mc)
            
        mdata = {}
        for d in mdata_list:
            mdata.update(d)
            
        mcolumns = [e for lst in mcolumns_list for e in lst]
        return DataFrame(mdata, columns=mcolumns)

    else:   
        mdata, mcolumns =  _melt(frame, id_vars=id_vars, value_vars=value_vars,
                             var_name=var_name, value_name=value_name,
                             col_level=col_level, stubnames=stubnames,
                             suffix=suffix, sep=sep)
        return DataFrame(mdata, columns=mcolumns)

### Main enhancement
Ability to simultaneously melt multiple groups of columns. Pass in a list of lists to `value_vars`. Importantly, groups do not have to be the same size

In [2]:
df = pd.DataFrame({'City': ['Houston', 'Austin', 'Hoover'],
                   'State': ['Texas', 'Texas', 'Alabama'],
                   'Name':['Aria', 'Penelope', 'Niko'],
                   'Mango':[4, 10, 90],
                   'Orange': [10, 8, 14], 
                   'Watermelon':[40, 99, 43],
                   'Gin':[16, 200, 34],
                   'Vodka':[20, 33, 18]},
                 columns=['City', 'State', 'Name', 'Mango', 'Orange', 'Watermelon', 'Gin', 'Vodka'])

df_large = df.sample(100000, replace=True)
df


,City,State,Name,Mango,Orange,Watermelon,Gin,Vodka
0,Houston,Texas,Aria,4,10,40,16,20
1,Austin,Texas,Penelope,10,8,99,200,33
2,Hoover,Alabama,Niko,90,14,43,34,18


In [3]:
melt(df, id_vars=['City', 'State'], value_vars=[['Mango', 'Orange', 'Watermelon'], ['Gin', 'Vodka']], 
     var_name=['Fruit', 'Drink'], value_name=['Pounds', 'Ounces'])

,City,State,Fruit,Pounds,Drink,Ounces
0,Houston,Texas,Mango,4,Gin,16.0
1,Austin,Texas,Mango,10,Gin,200.0
2,Hoover,Alabama,Mango,90,Gin,34.0
3,Houston,Texas,Orange,10,Vodka,20.0
4,Austin,Texas,Orange,8,Vodka,33.0
5,Hoover,Alabama,Orange,14,Vodka,18.0
6,Houston,Texas,Watermelon,40,nan,NaN
7,Austin,Texas,Watermelon,99,nan,NaN
8,Hoover,Alabama,Watermelon,43,nan,NaN


The current version of melt requires two separate melts and then a joining of the resulting frames. `pd.lreshape` cannot handle different sized groups.

In [4]:
try:
    pd.lreshape(df, {'temp':['yellow', 'green'], 'city':['FL']})
except Exception as e:
    print(type(e), e)

<class 'ValueError'> All column lists must be same length


`lreshape` only works with equal-sized groups

In [5]:
pd.lreshape(df, {'Fruit':['Mango', 'Orange'], 'Drink':['Gin', 'Vodka']})

,City,Name,State,Watermelon,Fruit,Drink
0,Houston,Aria,Texas,40,4,16
1,Austin,Penelope,Texas,99,10,200
2,Hoover,Niko,Alabama,43,90,34
3,Houston,Aria,Texas,40,10,20
4,Austin,Penelope,Texas,99,8,33
5,Hoover,Niko,Alabama,43,14,18


Slightly worse performance that `lreshape` but much better performance than the old `melt`

In [6]:
%%timeit 
melt(df, id_vars=['City', 'State'], value_vars=[['Mango', 'Orange'], ['Gin', 'Vodka']], 
     var_name=['Fruit', 'Drink'], value_name=['Pounds', 'Ounces'])

1.3 ms ± 51.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
%timeit pd.lreshape(df, {'Fruit':['Mango', 'Orange'], 'Drink':['Gin', 'Vodka']})

984 µs ± 34.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Better performance against old melt

In [8]:
melt(df, id_vars=['City', 'State'], value_vars=['Orange', 'Watermelon', 'Mango'], 
     var_name='Fruit', value_name='Pounds')

,City,State,Fruit,Pounds
0,Houston,Texas,Orange,10
1,Austin,Texas,Orange,8
2,Hoover,Alabama,Orange,14
3,Houston,Texas,Watermelon,40
4,Austin,Texas,Watermelon,99
5,Hoover,Alabama,Watermelon,43
6,Houston,Texas,Mango,4
7,Austin,Texas,Mango,10
8,Hoover,Alabama,Mango,90


In [9]:
%%timeit 
melt(df, id_vars=['City', 'State'], value_vars=['Orange', 'Watermelon', 'Mango'], 
     var_name='Fruit', value_name='Pounds')

962 µs ± 22.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%%timeit 
pd.melt(df, id_vars=['City', 'State'], value_vars=['Orange', 'Watermelon', 'Mango'], 
         var_name='Fruit', value_name='Pounds')

1.79 ms ± 51 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### wide_to_long functionality
wide_to_long functionality was not exactly replicated. Instead of merging the suffixes into one columns they are kept separate

In [11]:
df1 = pd.DataFrame({'group': ['a', 'b', 'c'],
                   'exp_1':[4, 10, -9],
                   'exp_2': [10, 8, 14], 
                   'res_1':[8, 5, 4],
                   'res_3':[11, 0, 7]}, columns=['group', 'exp_1', 'exp_2', 'res_1', 'res_3'])
df1

,group,exp_1,exp_2,res_1,res_3
0,a,4,10,8,11
1,b,10,8,5,0
2,c,-9,14,4,7


In [12]:
melt(df1, id_vars='group', value_vars=['exp','res'], stubnames=True, sep='_')

,group,variable_exp,exp,variable_res,res
0,a,1,4,1,8
1,b,1,10,1,5
2,c,1,-9,1,4
3,a,2,10,3,11
4,b,2,8,3,0
5,c,2,14,3,7


In [13]:
pd.wide_to_long(df1, stubnames=['exp', 'res'], i='group', j='num', sep='_')

exp   res
group num            
a     1     4.0   8.0
      2    10.0   NaN
      3     NaN  11.0
b     1    10.0   5.0
      2     8.0   NaN
      3     NaN   0.0
c     1    -9.0   4.0
      2    14.0   NaN
      3     NaN   7.0

Performance is much better

In [14]:
%timeit melt(df1, id_vars='group', value_vars=['exp','res'], stubnames=True, sep='_')

1.66 ms ± 25.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
%timeit pd.wide_to_long(df1, stubnames=['exp', 'res'], i='group', j='num', sep='_')

12.1 ms ± 203 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## MultiIndex Support
There was not much support in the original melt for MultiIndex DataFrames. This new melt allows for any number of column level selection.

In [16]:
df2 = df.copy()
df2.columns = pd.MultiIndex.from_arrays([list('aabbcccd'), list('ffffgggg'), df.columns], 
                                       names=[None, None, 'some vars'])
df2

a                  b            c                     d
                 f                  f            g                     g
some vars     City    State      Name Mango Orange Watermelon  Gin Vodka
0          Houston    Texas      Aria     4     10         40   16    20
1           Austin    Texas  Penelope    10      8         99  200    33
2           Hoover  Alabama      Niko    90     14         43   34    18

In [17]:
melt(df2, id_vars=[('a', 'f', 'State')], value_vars=[('b', 'f', 'Name'), ('c', 'g', 'Watermelon')])

,"(a, f, State)",variable_0,variable_1,some vars,value
0,Texas,b,f,Name,Aria
1,Texas,b,f,Name,Penelope
2,Alabama,b,f,Name,Niko
3,Texas,c,g,Watermelon,40
4,Texas,c,g,Watermelon,99
5,Alabama,c,g,Watermelon,43


Slight break in the API, uses column level if available.

In [18]:
pd.melt(df2, id_vars=[('a', 'f', 'State')], value_vars=[('b', 'f', 'Name'), ('c', 'g', 'Watermelon')])

,"(a, f, State)",variable_0,variable_1,variable_2,value
0,Texas,b,f,Name,Aria
1,Texas,b,f,Name,Penelope
2,Alabama,b,f,Name,Niko
3,Texas,c,g,Watermelon,40
4,Texas,c,g,Watermelon,99
5,Alabama,c,g,Watermelon,43


In [19]:
melt(df2, id_vars=[('a', 'f', 'State')], value_vars=[[('b', 'f', 'Name'), ('c', 'g', 'Watermelon')],
                                                   [('b','f','Mango'), ('c','g', 'Orange'), ('d', 'g', 'Vodka')]])

,"(a, f, State)",variable_0_0,variable_1_0,some vars_0,value_0,variable_0_1,variable_1_1,some vars_1,value_1
0,Texas,b,f,Name,Aria,b,f,Mango,4
1,Texas,b,f,Name,Penelope,b,f,Mango,10
2,Alabama,b,f,Name,Niko,b,f,Mango,90
3,Texas,c,g,Watermelon,40,c,g,Orange,10
4,Texas,c,g,Watermelon,99,c,g,Orange,8
5,Alabama,c,g,Watermelon,43,c,g,Orange,14
6,Texas,nan,nan,nan,NaN,d,g,Vodka,20
7,Texas,nan,nan,nan,NaN,d,g,Vodka,33
8,Alabama,nan,nan,nan,NaN,d,g,Vodka,18


Renaming multiple levels with multiple groups requires var_name to be a list of lists where the number of inner lists matches the number of groups and the lengths of the inner lists match the number of column levels.

In [20]:
melt(df2, id_vars=[('a', 'f', 'State')], value_vars=[[('b', 'f', 'Name'), ('c', 'g', 'Watermelon')],
                                                   [('b','f','Mango'), ('c','g', 'Orange'), ('d', 'g', 'Vodka')]],
    var_name=[['myvar1', 'myvar2', 'myvar3'],['next_myvar1', 'next_myvar2', 'next_myvar3']],
    value_name=['some values', 'more_values'])

,"(a, f, State)",myvar1,myvar2,myvar3,some values,next_myvar1,next_myvar2,next_myvar3,more_values
0,Texas,b,f,Name,Aria,b,f,Mango,4
1,Texas,b,f,Name,Penelope,b,f,Mango,10
2,Alabama,b,f,Name,Niko,b,f,Mango,90
3,Texas,c,g,Watermelon,40,c,g,Orange,10
4,Texas,c,g,Watermelon,99,c,g,Orange,8
5,Alabama,c,g,Watermelon,43,c,g,Orange,14
6,Texas,nan,nan,nan,NaN,d,g,Vodka,20
7,Texas,nan,nan,nan,NaN,d,g,Vodka,33
8,Alabama,nan,nan,nan,NaN,d,g,Vodka,18


### Choose any subset of column levels

In [21]:
melt(df2, id_vars=[('a', 'City')], col_level=[0, 'some vars']).head(10)

,"(a, City)",variable,some vars,value
0,Houston,a,State,Texas
1,Austin,a,State,Texas
2,Hoover,a,State,Alabama
3,Houston,b,Name,Aria
4,Austin,b,Name,Penelope
5,Hoover,b,Name,Niko
6,Houston,b,Mango,4
7,Austin,b,Mango,10
8,Hoover,b,Mango,90
9,Houston,c,Orange,10


In [22]:
melt(df2, id_vars=['Mango', 'Watermelon', 'Orange'], col_level=2).head()

,Mango,Watermelon,Orange,some vars,value
0,4,40,10,City,Houston
1,10,99,8,City,Austin
2,90,43,14,City,Hoover
3,4,40,10,State,Texas
4,10,99,8,State,Texas
